## Breast_Cancer_Prediction_Using_NN_Module

In [98]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

In [99]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [100]:
df.drop(columns = ['id', 'Unnamed: 32'], inplace= True)
df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [101]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,2:],df.iloc[:,0], train_size= 0.2)


In [102]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [103]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [104]:
X_train = torch.from_numpy(X_train).float()
X_test = torch.from_numpy(X_test).float()
y_train = torch.from_numpy(y_train).float()
y_test = torch.from_numpy(y_test).float()

In [105]:
X_train.dtype

torch.float32

In [106]:
learning_parameter = 0.1
epochs = 1000

## Feed Forward NN with Two Hidden Layer

In [107]:
class model(nn.Module):
    def __init__(self,X_size):
        super().__init__()
        self.layer1 = nn.Linear(X_size, 10)
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(10,5)
        self.layer3 = nn.Linear(5,1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, X_data):
        output = self.layer1(X_data)
        output = self.relu(output)
        output = self.layer2(output)
        output = self.relu(output)
        output = self.layer3(output)
        output = self.sigmoid(output)
        return output


In [108]:
model = model(X_train.shape[1])
model

model(
  (layer1): Linear(in_features=29, out_features=10, bias=True)
  (relu): ReLU()
  (layer2): Linear(in_features=10, out_features=5, bias=True)
  (layer3): Linear(in_features=5, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [114]:
lossfunc = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(),lr = learning_parameter)

In [115]:
for epoch in range(epochs):
    #Forward pass
    y_pred = model(X_train)

    #Loss Calculation
    loss = lossfunc(y_pred, y_train.view(-1,1))

    #Clear Gradients
    optimizer.zero_grad()

    #Backward Pass
    loss.backward()

    #Update the weights
    optimizer.step()

    print(f'Epoch : {epoch+1} Loss {loss.item()}')


Epoch : 1 Loss 0.0009581673657521605
Epoch : 2 Loss 0.0009568012319505215
Epoch : 3 Loss 0.0009554385324008763
Epoch : 4 Loss 0.0009540962055325508
Epoch : 5 Loss 0.0009527389775030315
Epoch : 6 Loss 0.0009514082921668887
Epoch : 7 Loss 0.0009500806918367743
Epoch : 8 Loss 0.0009487339993938804
Epoch : 9 Loss 0.0009474074468016624
Epoch : 10 Loss 0.0009460783912800252
Epoch : 11 Loss 0.0009447329794056714
Epoch : 12 Loss 0.0009434257517568767
Epoch : 13 Loss 0.0009420934948138893
Epoch : 14 Loss 0.0009407951729372144
Epoch : 15 Loss 0.0009394861408509314
Epoch : 16 Loss 0.0009381650597788393
Epoch : 17 Loss 0.000936847529374063
Epoch : 18 Loss 0.0009355631191283464
Epoch : 19 Loss 0.000934253737796098
Epoch : 20 Loss 0.0009329735767096281
Epoch : 21 Loss 0.0009316783980466425
Epoch : 22 Loss 0.000930380541831255
Epoch : 23 Loss 0.0009290888556279242
Epoch : 24 Loss 0.0009278189390897751
Epoch : 25 Loss 0.0009265269036404788
Epoch : 26 Loss 0.0009252637973986566
Epoch : 27 Loss 0.000923

## Inference

In [116]:
with torch.no_grad():
    y_inf = model(X_test)
    y_inf = (y_inf>0.6).float()
    accuracy = (y_inf == y_test).float().mean()
    print(f'Accuracy we are getting {accuracy}')

Accuracy we are getting 0.5341547131538391


## Using Sequential Container

In [117]:
class model_seq(nn.Module):
    def __init__(self,X_size):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(X_size,10),
            nn.ReLU(),
            nn.Linear(10,5),
            nn.ReLU(),
            nn.Linear(5,1),
            nn.Sigmoid()
        )

    def forward(self, X_data):
        output = self.network(X_data)
        return output


In [119]:
cancer_model = model_seq(X_train.shape[1])

In [121]:
loss_function = nn.BCELoss()
optimizer = torch.optim.SGD(cancer_model.parameters(),lr=learning_parameter)

In [123]:
for epoch in range(epochs):
    y_pred = cancer_model(X_train)
    loss = loss_function(y_pred, y_train.view(-1,1))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(f'Epoch {epoch+1} is {loss}')

Epoch 1 is 0.678662121295929
Epoch 2 is 0.6759486794471741
Epoch 3 is 0.673360288143158
Epoch 4 is 0.6708701252937317
Epoch 5 is 0.6684625744819641
Epoch 6 is 0.6661117672920227
Epoch 7 is 0.6638314127922058
Epoch 8 is 0.6616173386573792
Epoch 9 is 0.6594539284706116
Epoch 10 is 0.6573513150215149
Epoch 11 is 0.6553272008895874
Epoch 12 is 0.6533494591712952
Epoch 13 is 0.6514149308204651
Epoch 14 is 0.6495433449745178
Epoch 15 is 0.6476947069168091
Epoch 16 is 0.6458677649497986
Epoch 17 is 0.6440603733062744
Epoch 18 is 0.6422558426856995
Epoch 19 is 0.6404387354850769
Epoch 20 is 0.6385939717292786
Epoch 21 is 0.6367208957672119
Epoch 22 is 0.6348367929458618
Epoch 23 is 0.6329504251480103
Epoch 24 is 0.631020724773407
Epoch 25 is 0.6290411353111267
Epoch 26 is 0.627033531665802
Epoch 27 is 0.6249668598175049
Epoch 28 is 0.6228232383728027
Epoch 29 is 0.6206039190292358
Epoch 30 is 0.6183165907859802
Epoch 31 is 0.6159276962280273
Epoch 32 is 0.6134107708930969
Epoch 33 is 0.6107497

In [124]:
with torch.no_grad():
    y_pred = cancer_model(X_test)
    y_pred = (y_pred > 0.6).float()
    accuracy = (y_pred == y_test).float().mean()

print(accuracy)

tensor(0.5336)
